## H&M Personalized Fashion Recommendations

This notebook contains the approach taken for the 2022 H&M Personalized Fashion Recommendations Kaggle competition. 

*Visit repo README.md for instructions on how to execute notebook locally.*

Developed By **Jaileen Salazar**
_____

### Required Dependencies

In [15]:
from typing import Dict, Text

# Data Processing
import pandas as pd
import numpy as np
import csv

# Recommendation System
import tensorflow as tf
import tensorflow_recommenders as tfrs


### Constants

In [4]:
# FILE PATHS
TRANSACTIONS_PATH = '../data/transactions_train.csv'
CUSTOMER_PATH = '../data/customers.csv'
ARTICLES_PATH = '../data/articles.csv'
TEST_PATH = '../data/sample_submission.csv'

# FILE FORMATS
TRANSACTIONS_HEADERS = ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id']
ARTICLE_META_HEADERS = ['article_id', 'product_code', 'prod_name', 'product_type_no', 'product_type_name', 'product_group_name', 'graphical_appearance_no', 'graphical_appearance_name', 'colour_group_code', 'colour_group_name', 'perceived_colour_value_id', 'perceived_colour_value_name', 'perceived_colour_master_id', 'perceived_colour_master_name', 'department_no', 'department_name', 'index_code', 'index_name', 'index_group_no', 'index_group_name', 'section_no', 'section_name', 'garment_group_no', 'garment_group_name', 'detail_desc']
SUBMISSION_HEADERS = ['customer_id','prediction']
CUSTOMER_META_HEADERS = ['customer_id', 'fashion_news_frequency', 'age', 'postal_code']

# MODEL PARAMETERS
EMBEDDING_DIM = 32
EPOCHS = 3
BATCH_SIZE = 100

### Utility Methods

In [5]:
def parse_data(filepath, headers=None):
    """
        Open file, apply preprocessing and return formatted dataframe
    """
    df_data = pd.read_csv(filepath, sep=',', header=0, usecols=headers)
    if filepath == CUSTOMER_PATH:
        df_data['fashion_news_frequency'] = df_data['fashion_news_frequency'].fillna('NONE')
        df_data['age'] = df_data['age'].fillna(99)
        df_data['postal_code'] = df_data['postal_code'].fillna(0)

    return df_data

def save_results(ids, predictions, filename, headers):
    """
        Save predictions to csv file
    """
    data = zip(ids, predictions)
    with open(filename, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
        writer.writerows(data)

### Fashion Recommendations Class

In [6]:
class FashionRecommendationsModel(tfrs.Model):
    def __init__(self, user_model, product_model):
        super().__init__()
        self.user_model: tf.keras.Model = user_model
        self.product_model: tf.keras.Model = product_model
        self.task: tf.keras.layers.Layer = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["customer_id"])
        # And pick out the product features and pass them into the product model,
        # getting embeddings back.
        positive_product_embeddings = self.product_model(features["article_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_product_embeddings)

### Parse and Prepare Data

In [7]:
transactions = parse_data(filepath=TRANSACTIONS_PATH, headers=TRANSACTIONS_HEADERS)
# For training time purposes, only certain time period will be used
# Last transaction date is Sep 20 2021
transactions = transactions[transactions['t_dat'] >= '2020-09-01']

products = parse_data(filepath=ARTICLES_PATH, headers=ARTICLE_META_HEADERS)
products['detail_desc'] = products['detail_desc'].fillna('')
users = parse_data(filepath=CUSTOMER_PATH)
test_ids = parse_data(filepath=TEST_PATH, headers=['customer_id'])

### Create training and testing datasets

In [8]:
# Training -> 80,000
# Testing -> 20,000
transactions = transactions.sort_values(by=['t_dat'])
train_ds = tf.data.Dataset.from_tensor_slices(dict(transactions[['customer_id','article_id']].astype(str))).take(80_000)
test_ds = tf.data.Dataset.from_tensor_slices(dict(transactions[['customer_id','article_id']].astype(str))).skip(80_000).take(20_000)

# Remove any duplicates from user_ids and product_ids
user_ids = np.unique(users[['customer_id']])
product_ids = np.unique(products[['article_id']]).astype(str)

# Generate product Dataset for model
products_ds = tf.data.Dataset.from_tensor_slices(dict(products[['article_id']].astype(str)))
product_map = products_ds.map(lambda x: x['article_id'])

2022-05-08 02:55:23.491115: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Define Models

In [9]:
# Convert user_ids to ints for matrix factorization
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=user_ids, mask_token=None),
  # Additional embeddings to account for unknown tokens.
  tf.keras.layers.Embedding(len(user_ids) + 1, EMBEDDING_DIM)
])

# Convert product_ids to ints for matrix factorization
product_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=product_ids, mask_token=None),
  tf.keras.layers.Embedding(len(product_ids) + 1, EMBEDDING_DIM)
])

### Define Metrics

In [10]:
# Metrics: Compare affinity score that the model calculates for this pair to the scores of all the other possible candidates: if the score for the positive pair is higher than for all other candidates, our model is highly accurate.
metrics = tfrs.metrics.FactorizedTopK(
  candidates=product_map.batch(BATCH_SIZE).map(product_model)
)

# Loss
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

### Build Model

In [11]:
rec_model = FashionRecommendationsModel(user_model, product_model)
rec_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

### Fit and Evaluate Model

In [12]:
cached_train = train_ds.shuffle(100_000).batch(8192).cache()
cached_test = test_ds.batch(4096).cache()

In [13]:
rec_model.fit(cached_train, epochs=EPOCHS)

Epoch 1/3
10/10 [==============================] - 149s 15s/step - factorized_top_k/top_1_categorical_accuracy: 7.2500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0193 - factorized_top_k/top_10_categorical_accuracy: 0.0299 - factorized_top_k/top_50_categorical_accuracy: 0.0609 - factorized_top_k/top_100_categorical_accuracy: 0.0760 - loss: 70323.5554 - regularization_loss: 0.0000e+00 - total_loss: 70323.5554
Epoch 2/3
10/10 [==============================] - 146s 15s/step - factorized_top_k/top_1_categorical_accuracy: 0.0024 - factorized_top_k/top_5_categorical_accuracy: 0.1141 - factorized_top_k/top_10_categorical_accuracy: 0.1631 - factorized_top_k/top_50_categorical_accuracy: 0.3150 - factorized_top_k/top_100_categorical_accuracy: 0.4044 - loss: 68997.2202 - regularization_loss: 0.0000e+00 - total_loss: 68997.2202
Epoch 3/3
10/10 [==============================] - 150s 15s/step - factorized_top_k/top_1_categorical_accuracy: 0.0062 - factorized_top_k/top_5_categorical_accura

In [14]:
rec_model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 34s 7s/step - factorized_top_k/top_1_categorical_accuracy: 0.0030 - factorized_top_k/top_5_categorical_accuracy: 0.0099 - factorized_top_k/top_10_categorical_accuracy: 0.0145 - factorized_top_k/top_50_categorical_accuracy: 0.0419 - factorized_top_k/top_100_categorical_accuracy: 0.0645 - loss: 32536.8184 - regularization_loss: 0.0000e+00 - total_loss: 32536.8184


{'factorized_top_k/top_1_categorical_accuracy': 0.0030499999411404133,
 'factorized_top_k/top_5_categorical_accuracy': 0.00989999994635582,
 'factorized_top_k/top_10_categorical_accuracy': 0.014499999582767487,
 'factorized_top_k/top_50_categorical_accuracy': 0.041850000619888306,
 'factorized_top_k/top_100_categorical_accuracy': 0.06445000320672989,
 'loss': 29571.80078125,
 'regularization_loss': 0,
 'total_loss': 29571.80078125}

### Generate Predictions

In [16]:
# ScaNN used for its increased performance with large datasets
retrieval_index = tfrs.layers.factorized_top_k.ScaNN(rec_model.user_model, k=12)

# recommends products out of the entire transactions dataset.
retrieval_index = tfrs.layers.factorized_top_k.ScaNN(rec_model.user_model, k=12).index_from_dataset(
  tf.data.Dataset.zip((product_map.batch(100), product_map.batch(100).map(rec_model.product_model)))
)

# Get recommendations.
_, raw_preds = retrieval_index(test_ids.customer_id.values)
product_predications = raw_preds.numpy().astype(str)
product_predications = pd.Series(map(' '.join, product_predications))
save_results(test_ids.customer_id.values, product_predications, filename='submission.csv', headers=SUBMISSION_HEADERS)


: 

: 